### 양돈 월별 데이터 수집

In [15]:
import pandas as pd

df_monthly = pd.read_excel('../data/pig_monthly.xlsx', header=1)
df_monthly = df_monthly.drop(columns=['지역'])
df_monthly.head()

/home/jykim/venv/eda_venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,년도,규모,농장,데이터구분,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2020,100~200,PIGGO_156,분만율,81.3,81.1,73.9,76.6,80.6,68.6,79.4,77.4,75.9,79.4,72.2,75.0
1,2019,100~200,PIGGO_156,평균이유,9.0,8.5,9.9,9.0,9.1,9.0,9.1,9.7,8.6,9.0,9.5,10.1
2,2018,100~200,PIGGO_156,임신사고(불규칙),0.0,0.0,2.0,3.0,1.0,1.0,2.0,3.0,4.0,2.0,4.0,2.0
3,2017,100~200,PIGGO_156,7일내재귀율,87.0,83.3,87.5,78.9,78.9,90.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,100~200,PIGGO_156,임신사고(도태),NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
import numpy as np

df_monthly = df_monthly.replace({np.nan: None})

In [17]:
df_monthly

,년도,규모,농장,데이터구분,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2020,100~200,PIGGO_156,분만율,81.3,81.1,73.9,76.6,80.6,68.6,79.4,77.4,75.9,79.4,72.2,75.0
1,2019,100~200,PIGGO_156,평균이유,9.0,8.5,9.9,9.0,9.1,9.0,9.1,9.7,8.6,9.0,9.5,10.1
2,2018,100~200,PIGGO_156,임신사고(불규칙),0.0,0.0,2.0,3.0,1.0,1.0,2.0,3.0,4.0,2.0,4.0,2.0
3,2017,100~200,PIGGO_156,7일내재귀율,87.0,83.3,87.5,78.9,78.9,90.0,None,None,None,None,None,None
4,2017,100~200,PIGGO_156,임신사고(도태),None,None,None,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16221,2018,1000두 이상,PIGGO_25,임신사고(불임),4.0,1.0,0.0,1.0,1.0,2.0,1.0,2.0,4.0,0.0,1.0,0.0
16222,2017,1000두 이상,PIGGO_25,임신사고(1차),86.0,65.0,35.0,22.0,34.0,48.0,65.0,83.0,87.0,67.0,82.0,61.0
16223,2016,1000두 이상,PIGGO_25,임신사고(공태),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16224,2015,1000두 이상,PIGGO_25,임신사고(공태),0.0,None,0.0,0.0,None,None,None,None,None,None,None,0.0


In [18]:
import mysql.connector

conn = mysql.connector.connect(
    host = 'database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'kjy',
    password = '0000',
    database = 'pig'
)

cursor = conn.cursor(buffered=True)

In [19]:
sql = '''
create table pig_db_monthly
(
    id int not null primary key,
    year int,
    size varchar(16),
    data_dict varchar(16),
    farm_id varchar(16),
    jan float,
    feb float,
    mar float,
    apr float,
    may float,
    jun float,
    jul float,
    aug float,
    sep float,
    oct float,
    nov float,
    december float    
)
'''
cursor.execute(sql)

In [20]:
sql = '''
insert into pig_db_monthly
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

for i, row in df_monthly.iterrows():
    value = (i,) + tuple(row)
    cursor.execute(sql, value)
    if i <= 5:
        print(tuple(row))
    conn.commit()


(2020, '100~200', 'PIGGO_156', '분만율', 81.3, 81.1, 73.9, 76.6, 80.6, 68.6, 79.4, 77.4, 75.9, 79.4, 72.2, 75.0)
(2019, '100~200', 'PIGGO_156', '평균이유', 9.0, 8.5, 9.9, 9.0, 9.1, 9.0, 9.1, 9.7, 8.6, 9.0, 9.5, 10.1)
(2018, '100~200', 'PIGGO_156', '임신사고(불규칙)', 0.0, 0.0, 2.0, 3.0, 1.0, 1.0, 2.0, 3.0, 4.0, 2.0, 4.0, 2.0)
(2017, '100~200', 'PIGGO_156', '7일내재귀율', 87.0, 83.3, 87.5, 78.9, 78.9, 90.0, None, None, None, None, None, None)
(2017, '100~200', 'PIGGO_156', '임신사고(도태)', None, None, None, None, None, None, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(2016, '100~200', 'PIGGO_156', '모돈회전율', 2.19, 2.03, 1.91, 1.94, 2.16, 2.31, 2.3, 2.4, 2.42, 2.38, 2.44, 2.87)


In [21]:
cursor.execute('select count(*) from pig_db_monthly')
result = cursor.fetchall()
print('table에 삽입한 데이터의 길이:',result[0][0])
print('원본 데이터의 길이:',len(df_monthly))

table에 삽입한 데이터의 길이: 16226
원본 데이터의 길이: 16226


### 양돈 년도별 데이터 수집

In [22]:
import pandas as pd

df_yearly = pd.read_excel('../data/pig_yearly.xlsx', header=1)
df_yearly = df_yearly.drop(columns=['지역'])
df_yearly.head()

/home/jykim/venv/eda_venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,년도,규모,농장,상시모돈수(두),MSY(두),PSY(두),모돈회전율(회전),평균비생산일수(일),평균총산(두),평균실산(두),평균이유(두),분만율(%),초교배일령(일),임신기간(일),포유기간(일),재귀발정일(일)
0,2019,100~200,PIGGO_156,143.7,17.2,19.7,2.13,74.6,13.0,11.0,9.0,65.4,256.7,115.9,20.4,7.7
1,2021,500~1000,PIGGO_182,523.4,16.2,24.9,2.28,45.6,14.0,12.0,11.0,82.4,261.1,115.0,26.8,7.7
2,2018,100~200,PIGGO_156,146.1,19.2,21.7,2.18,69.4,13.0,12.0,10.0,76.3,263.1,116.0,20.2,12.9
3,2021,500~1000,0021749,636.9,NaN,29.2,2.52,25.3,14.0,12.0,12.0,90.5,268.5,115.5,23.6,7.9
4,2021,400~500,0021437,453.6,NaN,30.9,2.29,43.9,16.0,14.0,14.0,81.5,232.3,114.3,27.1,8.4


In [23]:
import numpy as np

df_yearly = df_yearly.replace({np.nan: None})
df_yearly

,년도,규모,농장,상시모돈수(두),MSY(두),PSY(두),모돈회전율(회전),평균비생산일수(일),평균총산(두),평균실산(두),평균이유(두),분만율(%),초교배일령(일),임신기간(일),포유기간(일),재귀발정일(일)
0,2019,100~200,PIGGO_156,143.7,17.2,19.7,2.13,74.6,13.0,11.0,9.0,65.4,256.7,115.9,20.4,7.7
1,2021,500~1000,PIGGO_182,523.4,16.2,24.9,2.28,45.6,14.0,12.0,11.0,82.4,261.1,115.0,26.8,7.7
2,2018,100~200,PIGGO_156,146.1,19.2,21.7,2.18,69.4,13.0,12.0,10.0,76.3,263.1,116.0,20.2,12.9
3,2021,500~1000,0021749,636.9,None,29.2,2.52,25.3,14.0,12.0,12.0,90.5,268.5,115.5,23.6,7.9
4,2021,400~500,0021437,453.6,None,30.9,2.29,43.9,16.0,14.0,14.0,81.5,232.3,114.3,27.1,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,2013,500~1000,PIGGO_346,682.5,19.7,25.2,2.5,25.7,13.0,11.0,10.0,89.2,259.8,114.9,23.1,5.8
1058,2013,500~1000,PIGGO_244,775.5,None,25.3,2.38,34.0,13.0,12.0,11.0,84.8,263.8,115.1,24.4,5.8
1059,2013,1000두 이상,PIGGO_15,1549.3,None,20.2,2.13,79.4,12.0,11.0,10.0,59.1,264.7,115.7,24.3,8.9
1060,2013,1000두 이상,PIGGO_16,1667.7,None,24.0,2.41,40.4,11.0,10.0,10.0,78.7,273.1,115.3,20.5,6.7


In [24]:
sql = '''
create table pig_db_yearly
(
    id int not null primary key,
    year int,    
    size varchar(16),
    farm_id varchar(16),
    sow_inventory int,
    msy float,
    psy float,
    sow_turnover float,
    nonprod_days int,
    total_born_avg int,
    live_born_avg int,
    weand_avg int,
    farrowing_rate float,
    first_mating_age int,
    gestation_len int,
    lactation_period int,
    return_estrus_days int   
)
'''
cursor.execute(sql)

In [25]:
sql = '''
insert into pig_db_yearly
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

for i, row in df_yearly.iterrows():
    value = (i,) + tuple(row)
    cursor.execute(sql, value)
    if i <= 5:
        print(tuple(row))
    conn.commit()

(2019, '100~200', 'PIGGO_156', 143.7, 17.2, 19.7, 2.13, 74.6, 13.0, 11.0, 9.0, 65.4, 256.7, 115.9, 20.4, 7.7)
(2021, '500~1000', 'PIGGO_182', 523.4, 16.2, 24.9, 2.28, 45.6, 14.0, 12.0, 11.0, 82.4, 261.1, 115.0, 26.8, 7.7)
(2018, '100~200', 'PIGGO_156', 146.1, 19.2, 21.7, 2.18, 69.4, 13.0, 12.0, 10.0, 76.3, 263.1, 116.0, 20.2, 12.9)
(2021, '500~1000', '0021749', 636.9, None, 29.2, 2.52, 25.3, 14.0, 12.0, 12.0, 90.5, 268.5, 115.5, 23.6, 7.9)
(2021, '400~500', '0021437', 453.6, None, 30.9, 2.29, 43.9, 16.0, 14.0, 14.0, 81.5, 232.3, 114.3, 27.1, 8.4)
(2021, '200~300', '0021398', 251.5, None, 28.6, 2.48, 30.6, 14.0, 13.0, 12.0, 88.0, 254.7, 114.1, 21.2, 6.0)


In [26]:
cursor.execute('select count(*) from pig_db_yearly')
result = cursor.fetchall()
print('table에 삽입한 데이터의 길이:',result[0][0])
print('원본 데이터의 길이:',len(df_yearly))

table에 삽입한 데이터의 길이: 1062
원본 데이터의 길이: 1062


In [27]:
conn.close()